<a href="https://colab.research.google.com/github/hafsatariq18/LangChain/blob/main/zilliz_rag_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai pypdf bs4


In [ ]:
!pip install pymupdf



In [4]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass()


··········


In [6]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("Story.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

docs[1]



Document(metadata={'source': 'Story.pdf', 'file_path': 'Story.pdf', 'page': 0, 'total_pages': 3, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'pdfmake', 'producer': 'pdfmake', 'creationDate': 'D:20240902104121Z', 'modDate': '', 'trapped': ''}, page_content='friends were Ali, Kabir, Zoya, and Sara.\n \nAli was the eldest among them and the most responsible. He worked tirelessly as a farmer, tending to his small patch of land with dedication \nand care. Life as a farmer was difficult, with unpredictable weather and meager harvests, but Ali never complained. He was a natural leader, \nalways looking out for his friends and family, and his wisdom guided the group through many challenges. His dream was simple—to one day')

In [15]:
from langchain_milvus import Milvus, Zilliz
from langchain_openai import OpenAIEmbeddings
from getpass import getpass

os.environ["URI"] = getpass("Enter the Milvus/Zilliz URI: ")
os.environ["TOKEN"] = getpass("Enter the Zilliz Token: ")

uri = os.getenv("URI")
token = os.getenv("TOKEN")

embeddings = OpenAIEmbeddings()

vectorstore = Milvus.from_documents(  # or Zilliz.from_documents
    documents=docs,
    embedding=embeddings,
    connection_args={
        "uri": uri,
        "token": token,
    },
    drop_old=True,  # Drop the old Milvus collection if it exists
)



Enter the Milvus/Zilliz URI: ··········
Enter the Zilliz Token: ··········


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 9078ab4705084d63bf47e2a19f38df1d


In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

PROMPT_TEMPLATE = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use two sentences maximum and keep the "
        "answer concise.\n\n{context}"
    )

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
retriever = vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [22]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


res = rag_chain.invoke("Who is kabir")
res


'Kabir ran a small tea stall in the village, where he made people feel welcome and valued. Ali, the eldest among the friends, worked as a farmer and dreamed of owning enough land to provide for his aging parents.'